### 原理概述
1. 已知$n$个带标签的样本点
2. 新来一个点$x$，通过计算每个样本点与点$x$之间的距离，找出与该点最近的$k$个样本点（$k$一般不超过20）
3. 统计这$k$个点中出现的标签及其频数，选取频数最高的标签贴给新点$x$

In [19]:
from numpy import *
import operator

def createDataSet():
    group = array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]])
    labels = ['A','A','B','B']
    return group, labels

group,labels=createDataSet()

In [20]:
group

array([[ 1. ,  1.1],
       [ 1. ,  1. ],
       [ 0. ,  0. ],
       [ 0. ,  0.1]])

In [21]:
labels

['A', 'A', 'B', 'B']

In [22]:
def classify0(inX, dataSet, labels, k):
    dataSetSize = dataSet.shape[0] #获取样本集大小
    diffMat = tile(inX, (dataSetSize,1))-dataSet #把输入点复制成4份，与样本集相减得出输入点与每个样本点之间的△x和△y
    sqDiffMat = diffMat**2 #求△x的平方和△y的平方
    sqDistances = sqDiffMat.sum(axis=1) #相当于两个列向量相加，每个列向量的两个元素是△x的平方和△y的平方
    #所得结果是含4个元素的列向量，各元素值为与四个样本点之间的距离的平方
    distances = sqDistances**0.5 #开方得距离
    sortedDistIndicies = distances.argsort() #排序，返回有序的索引值
    classCount={}
    for i in range(k):
        voteIlabel = labels[sortedDistIndicies[i]] #遍历拿前k个的标签
        classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1 #拿到的那个标签计数+1（如果该标签尚不存在与字典中，直接返回0）
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True) #Python3字典用items()迭代
    # 返回的sortedClassCount是个list，形如[('A', 2), ('B', 1)]
    return sortedClassCount[0][0]

In [23]:
#测试-四个已知点是否归类正确
res=[]
res.append(classify0([1. , 1.1],group,labels,3))
res.append(classify0([1. , 1. ],group,labels,3))
res.append(classify0([0. , 0. ],group,labels,3))
res.append(classify0([0. , 0.1],group,labels,3))
res

['A', 'A', 'B', 'B']

## classify0的分解说明
### 1. tile -------------------------------

In [24]:
import numpy as np
b = np.array([[1, 2], [3, 4]])
b

array([[1, 2],
       [3, 4]])

#### 横着堆2个

In [25]:
np.tile(b, 2)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

#### 竖着堆3个（由数字1指示）

In [26]:
np.tile(b, (3, 1))

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

###  2. sum -------------------------------

In [27]:
b = np.array([[1, 2], [3, 4]])
b

array([[1, 2],
       [3, 4]])

#### 所有元素之和

In [28]:
b.sum()

10

#### 竖着加

In [29]:
b.sum(axis=0)

array([4, 6])

#### 横着加

In [30]:
b.sum(axis=1)

array([3, 7])

### 3. argsort -------------------------------
#### 基本排序：返回排完后的索引值

In [31]:
x = np.array([3, 1, 2])
np.argsort(x) # 结果 array([1, 2, 0]) 表示x[1]<x[2]<x[0]. 写成x.argsort()也行

array([1, 2, 0], dtype=int64)

#### 矩阵元素排序

In [32]:
x = np.array([[0, 3], [4, 2]])
print(x)
print(np.argsort(x, axis=0)) #横排
print(np.argsort(x, axis=1)) #竖排

[[0 3]
 [4 2]]
[[0 1]
 [1 0]]
[[0 1]
 [1 0]]


### 4. 字典的get() -------------------------------

In [33]:
d={'a':4,'b':6}
d.get('a')

4

#### 如果索引不在字典中，返回第二个参数

In [34]:
d.get('c',0)

0

### 5. 字典的items() -------------------------------

In [35]:
d.items()

dict_items([('a', 4), ('b', 6)])

### 6. sorted() -------------------------------
```
sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
```
- 意为对classCount中所有元素排序
- key=operator.itemgetter(1)表示顺序将按“每个被排序元素”的第1域的值来排
 - 每个被排序元素 都是形如 ('A',2) 的元组，第0域是'a'，第1域是2
- reverse=True表明是按从大到小的顺序排列

### 7. operator.itemgetter(1) -------------------------------

In [36]:
a = [1,2,3] 
b=operator.itemgetter(1) #定义函数b，获取对象的第1个域的值
b(a)

2

In [37]:
b=operator.itemgetter(1,0) #定义函数b，获取对象的第1个域和第0个的值
b(a)

(2, 1)